In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from transformers import DebertaTokenizer, DebertaForSequenceClassification,AutoTokenizer
from torchsummary import summary
import pandas as pd
import os

/Users/lemarx/anaconda3/envs/str23/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

True
True


In [37]:
class SentenceSimilarityModel(pl.LightningModule):
    def __init__(self, learning_rate=2e-5):
        super(SentenceSimilarityModel, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
        self.deberta = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-v3-small',ignore_mismatched_sizes=True,num_labels=1)
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask):
        outputs = self.deberta(input_ids, attention_mask=attention_mask)
        return torch.sigmoid(outputs.logits)

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits = self(input_ids, attention_mask)
        
        # Assuming a binary classification task
        loss = F.binary_cross_entropy_with_logits(logits, labels.float())
        
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits = self(input_ids, attention_mask)
        
        # Assuming a binary classification task
        loss = F.binary_cross_entropy_with_logits(logits, labels.float())
        
        # Assuming you want to track accuracy during validation
        predictions = torch.round(torch.sigmoid(logits))
        accuracy = (predictions == labels).sum().float() / labels.size(0)
        
        return {'val_loss': loss, 'val_accuracy': accuracy}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_accuracy = torch.stack([x['val_accuracy'] for x in outputs]).mean()
        self.log('val_loss', avg_loss)
        self.log('val_accuracy', avg_accuracy)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)


In [4]:
class SentenceSimilarityDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, batch_size=16):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)


In [5]:
train_data = pd.read_csv(os.path.join('..','data','raw','eng_train.csv'))
train_data.head()

,PairID,Text,Score
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0
3,ENG-train-0003,If he is good looking and has a good personali...,1.0
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0


In [6]:
sep = '[SEP]'
test = train_data['Text'].loc[0].replace('\n',sep)
test

'It that happens, just pull the plug.[SEP]if that ever happens, just pull the plug.'

In [20]:
train_data['input'] = train_data.apply(lambda row : row['Text'].replace('\n',sep),axis = 1)
train_data.head()

,PairID,Text,Score,input
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0,"It that happens, just pull the plug.[SEP]if th..."
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0,A black dog running through water.[SEP]A black...
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0,I've been searchingthe entire abbey for you.[S...
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,If he is good looking and has a good personali...
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,"She does not hate you, she is just annoyed wit..."


In [38]:
test = train_data['input'].loc[0]
model = SentenceSimilarityModel()
token = model.tokenizer(test, return_tensors= 'pt')
input_ids = token['input_ids']
attention_mask = token['attention_mask']

with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    logits = model(input_ids, attention_mask)


print("Logits:", logits)

/Users/lemarx/anaconda3/envs/str23/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
You are using a model of type deberta-v2 to instantiate a model of type deberta. This is not supported for all configurations of models and can yield errors.
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['deberta.encoder.layer.0.attention.self.q_bias', 'deberta.encoder.layer.3.attention.self.pos_q_proj.weight', 'deberta.encoder.layer.5.attention.self.q_bias', 'deberta

Logits: tensor([[0.4538]])


In [27]:
model.deberta.config.hidden_size

768

In [28]:
model.additional_layer

Linear(in_features=768, out_features=1, bias=True)